<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/gif_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1883, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1883 (delta 27), reused 0 (delta 0), pack-reused 1829
Receiving objects: 100% (1883/1883), 83.12 MiB | 10.88 MiB/s, done.
Resolving deltas: 100% (904/904), done.
Checking out files: 100% (1954/1954), done.


In [4]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [6]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_data.py

100% 5/5 [00:02<00:00,  1.95it/s]
{0: 'BEEM', 1: 'MCD', 2: 'PSTV', 3: 'RWB', 4: 'XXII'}
{'BEEM': 0, 'MCD': 1, 'PSTV': 2, 'RWB': 3, 'XXII': 4}
train labels:  (10908, 192)
train inputs:  (10908, 192, 5)
  0% 0/1151 [00:00<?, ?it/s]Company Name FCX not in Train Data. Hence skipping
Company Name FTCH not in Train Data. Hence skipping
Company Name TGT not in Train Data. Hence skipping
Company Name CRWD not in Train Data. Hence skipping
Company Name CRON not in Train Data. Hence skipping
Company Name MGM not in Train Data. Hence skipping
Company Name CRSM not in Train Data. Hence skipping
Company Name PRU not in Train Data. Hence skipping
Company Name CLOV not in Train Data. Hence skipping
Company Name BTCUSD not in Train Data. Hence skipping
Company Name PIK not in Train Data. Hence skipping
Company Name NEM not in Train Data. Hence skipping
Company Name FXE not in Train Data. Hence skipping
Company Name VRM not in Train Data. Hence skipping
Company Name FOMC not in Train Data. Hence skippi

In [7]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [8]:
print(index_to_company)
print(company_to_index)

{0: 'BEEM', 1: 'MCD', 2: 'PSTV', 3: 'RWB', 4: 'XXII'}
{'BEEM': 0, 'MCD': 1, 'PSTV': 2, 'RWB': 3, 'XXII': 4}


In [9]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [10]:
NUM_TRAIN_SAMPLES = 10908
NUM_TEST_SAMPLES  = 4236

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [11]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (10908, 192, 5)
stock_labels_trimmed shape (10908, 192)
stock_test_inputs_trimmed shape (4236, 192, 5)
stock_test_labels_trimmed shape (4236, 192)


In [12]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, NUM_TRAIN_SAMPLES):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, NUM_TEST_SAMPLES):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])




In [13]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))


[0.0, 1.0, 2.0, 3.0, 4.0]
5
[0.0, 1.0, 2.0, 4.0]
4


In [14]:

def generate_horizon_weights(horizon_len):


  # weight = [i for i in range(1, horizon_len + 1)]
  
  weight = [1]
  for i in range(1, horizon_len):
    weight.append(weight[-1]*2)

  weight_arr = np.array(weight)

  return weight_arr/ np.sum(weight_arr)

HORIZON_PERIOD = 8

params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':64,
    'cov_dim': 3,
    'lstm_hidden_dim': 128,
    'lstm_layers':3 ,
    'window_size':192,
    'batch_size': 300,
    'learning_rate': 1e-3,
    'epochs':30,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'horizon_period': HORIZON_PERIOD,
    'horizon_weights': generate_horizon_weights(HORIZON_PERIOD)
}

# print(hor_weight)

assert sum(params['horizon_weights']) == 1.0
assert len(params['horizon_weights']) == params['horizon_period']

print(params)

{'num_classes': 5, 'embedding_dim': 64, 'cov_dim': 3, 'lstm_hidden_dim': 128, 'lstm_layers': 3, 'window_size': 192, 'batch_size': 300, 'learning_rate': 0.001, 'epochs': 30, 'num_test_samples': 4236, 'num_train_samples': 10908, 'conditioning_period': 168, 'prediction_period': 24, 'horizon_period': 8, 'horizon_weights': array([0.00392157, 0.00784314, 0.01568627, 0.03137255, 0.0627451 ,
       0.1254902 , 0.25098039, 0.50196078])}


In [15]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):

        x = np.delete(self.data[index], 1, axis=1)
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2
        
class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
        x = np.delete(self.data[index], 1, axis=1)
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2
        

In [16]:
class TrainDatasetHorizon(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]
        self.horizon_period = params['horizon_period']  # Return (t+1, t+2, t+3, ...., t+5) into the future


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):

        x = np.delete(self.data[index], 1, axis=1)
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))

        x_shift = [None]*self.horizon_period

        x_shift = [torch.roll(x2, shifts=-1*i, dims=0)[:-1*(self.horizon_period - 1 )] for i in range(0, self.horizon_period)]

        # Retrun 0, 1, 2, 3, ......, num_samples - horizon_period samples
        # EG. Window Length of 192 (0-191), HORIZON Size of 5

        # IP Features will be from (0-187)

        # Note that the Original Dataset is already "1" shifted
        # There will be 5 Labels: 0-187 (1 shifted), 
        # 1-188 (2-shifted), 
        # 2-189 (3-shifted),
        # 3-190 (4-shifted),
        # 4-191 (5-shifted)
        
        return x1[:-1*(self.horizon_period - 1 ), :], x_shift

In [ ]:
# data_set = TrainDatasetHorizon()

# for x, x_shift in data_set:

#   print("x shape", x.shape, 'x shift', x_shift[0].shape)



In [17]:
def unwindow_sequence(dataset, data_type="normal"):
  company_data = {}

  seq_id = 0
  company_index_cov_id = -1

  for cov_ip, cov_original_op in dataset:

    if data_type == "train_horizon":

      cov_op = cov_original_op[0]
      # print("---first cov op ---", cov_op)
      # print("---appending with 0----", cov_original_op[-1][-1*params['horizon_period']:])
      # cov_op = cov_op.append(cov_original_op[-1][-1*params['horizon_period']:])
      cov_op = torch.cat((cov_op, cov_original_op[-1][-1*(params['horizon_period'] - 1):]), dim = 0) 

      # print("cov op shape", cov_op.shape)
    else:
      cov_op = cov_original_op

    comp_index = cov_ip[seq_id][company_index_cov_id]
    company_name = index_to_company[comp_index.item()]

    # IP Shape (Time, Dim)
    # OP Shape (Time, 1) # One covariate (open price) per time instant
    if company_data.get(company_name, None) is None:
      company_data[company_name] = [cov_ip, cov_op]
      
    else:
      # windows are created with stride 1
      # so we should be appending only the last element in the sequence
      cov_ip_last = cov_ip[-1, :].unsqueeze(0)
      cov_op_last = cov_op[-1].unsqueeze(0)

      # 0 --> Covariates
      # 1 --> Outputs, aka stock price
      company_data[company_name][0] = torch.cat((company_data[company_name][0], cov_ip_last), dim=0)
      company_data[company_name][1] = torch.cat((company_data[company_name][1], cov_op_last), dim=0)

  return company_data

In [18]:
train_data = TrainDatasetHorizon()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)

# train_data_unwindowed = unwindow_sequence(train_data, data_type="train_horizon")
# test_data_unwindowed = unwindow_sequence(test_data)


In [ ]:
# print("train_data_unwindowed", train_data_unwindowed.shape)
# print("test_data_unwindowed", test_data_unwindowed.shape)

In [ ]:
# print(train_data_unwindowed.keys())
# print(len(train_data_unwindowed))

NameError: ignored

In [ ]:
print(test_data_unwindowed.keys())
print(len(test_data_unwindowed))
# Some Companies are there in the Train Set, But not in the Test Set. 
# However, all companies in the Test Set is there in the Train Set

NameError: ignored

In [19]:
import plotly.express as px

def plot_output(company_data, company_name):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[company_name][1]), num=len(company_data[company_name][1]))
  fig = px.line(x=x, y=company_data[company_name][1])
  fig.show()
  print("op", company_data[company_name][1])

def plot_input_covariate(company_data, company_name, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[company_name][0].shape[0], num=len(company_data[company_name][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[company_name][0][:,covariate_index])
  fig.show()
  print("ip", company_data[company_name][0][:,covariate_index])

In [21]:
# SANITY Check One: Output Label (open price) is shifted by one element of input (the ip opening price)
plot_output(train_data_unwindowed, 'MCD')
plot_input_covariate(train_data_unwindowed, 'MCD', 0)

NameError: ignored

In [22]:

# SANITY Check TWO: Train and Test data are contiguous
plot_input_covariate(train_data_unwindowed, 'MCD', 0)
# plot_input_covariate(test_data_unwindowed, 'MCD', 0)
plot_output(train_data_unwindowed, 'MCD')

NameError: ignored

In [23]:
# SANITY Check TWO for Output Labels: Train and Test data are contiguous

plot_output(train_data_unwindowed, 'MCD')
plot_output(test_data_unwindowed, 'MCD')

NameError: ignored

In [ ]:
plot_input_covariate(train_data_unwindowed, 'MCD', 0)  # Opening Price
# plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Volume
plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Day
plot_input_covariate(train_data_unwindowed, 'MCD', 2)  # Month
plot_input_covariate(train_data_unwindowed, 'MCD', 3)  # Company ID


In [20]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[window_id][1]), num=len(company_data[window_id][1]))
  fig = px.line(x=x, y=company_data[window_id][1])
  fig.show()
  print("op", company_data[window_id][1])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[window_id][0].shape[0], num=len(company_data[window_id][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][0][:,covariate_index])
  fig.show()
  print("ip", company_data[window_id][0][:,covariate_index])


In [25]:
plot_output_windowed(test_data, 0)
plot_input_covariate_windowed(test_data, 0, 0)

op tensor([-2.0504e+00, -2.0353e+00, -2.0538e+00, -1.9615e+00, -1.9474e+00,
        -1.9474e+00, -1.8663e+00, -1.8460e+00, -1.7911e+00, -1.8305e+00,
        -1.8668e+00, -1.8663e+00, -1.8663e+00, -1.8820e+00, -1.8742e+00,
        -1.9061e+00, -1.9061e+00, -1.8820e+00, -1.8927e+00, -2.0052e+00,
        -1.9334e+00, -1.9474e+00, -1.9343e+00, -1.9759e+00, -1.9759e+00,
        -1.9466e+00, -1.8663e+00, -1.8663e+00, -1.9759e+00, -1.9474e+00,
        -1.9474e+00, -1.9197e+00, -1.9615e+00, -1.9477e+00, -1.9904e+00,
        -1.9878e+00, -2.0445e+00, -2.0353e+00, -2.0727e+00, -1.9904e+00,
        -1.9474e+00, -1.9759e+00, -1.8927e+00, -2.0052e+00, -1.9587e+00,
        -1.9474e+00, -1.9673e+00, -1.9197e+00, -2.0353e+00, -2.0570e+00,
        -2.0648e+00, -2.0984e+00, -2.1314e+00, -2.3304e+00, -2.3761e+00,
        -2.1655e+00, -2.2372e+00, -2.4857e+00, -2.6405e+00, -2.5119e+00,
        -2.5871e+00, -2.5871e+00, -2.5871e+00, -2.5966e+00, -2.6682e+00,
        -2.6410e+00, -2.6339e+00, -2.6967e+00, -

ip tensor([-1.9474e+00, -2.0504e+00, -2.0353e+00, -2.0538e+00, -1.9615e+00,
        -1.9474e+00, -1.9474e+00, -1.8663e+00, -1.8460e+00, -1.7911e+00,
        -1.8305e+00, -1.8668e+00, -1.8663e+00, -1.8663e+00, -1.8820e+00,
        -1.8742e+00, -1.9061e+00, -1.9061e+00, -1.8820e+00, -1.8927e+00,
        -2.0052e+00, -1.9334e+00, -1.9474e+00, -1.9343e+00, -1.9759e+00,
        -1.9759e+00, -1.9466e+00, -1.8663e+00, -1.8663e+00, -1.9759e+00,
        -1.9474e+00, -1.9474e+00, -1.9197e+00, -1.9615e+00, -1.9477e+00,
        -1.9904e+00, -1.9878e+00, -2.0445e+00, -2.0353e+00, -2.0727e+00,
        -1.9904e+00, -1.9474e+00, -1.9759e+00, -1.8927e+00, -2.0052e+00,
        -1.9587e+00, -1.9474e+00, -1.9673e+00, -1.9197e+00, -2.0353e+00,
        -2.0570e+00, -2.0648e+00, -2.0984e+00, -2.1314e+00, -2.3304e+00,
        -2.3761e+00, -2.1655e+00, -2.2372e+00, -2.4857e+00, -2.6405e+00,
        -2.5119e+00, -2.5871e+00, -2.5871e+00, -2.5871e+00, -2.5966e+00,
        -2.6682e+00, -2.6410e+00, -2.6339e+00, -

In [ ]:
print('train data shape', train_data[0][0].shape)

train data shape torch.Size([179, 4])


# MODEL


In [21]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )


        self.distribution_premu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_mu = nn.GELU()
        self.distribution_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_sigma = nn.Softplus()


    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        onehot_embed = self.embedding(company_index)
        
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_premu = self.distribution_premu(out1)
        out_mu = self.distribution_mu(out_premu)

        out_presigma = self.distribution_presigma(out1)
        out_sigma = self.distribution_sigma(out_presigma)

        return out_mu, out_sigma, hn_cn



In [22]:
model = Network(params)

In [23]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_mu, out_sigma, ht_ct = model.forward(x=ip_covariate)

  print('out mu shape', out_mu.shape)
  print('out_sigma shape', out_sigma.shape)
  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

out mu shape torch.Size([300, 185, 1])
out_sigma shape torch.Size([300, 185, 1])
ht shape torch.Size([3, 300, 128]) torch.Size([3, 300, 128])


In [24]:
def loss_fn(mu,sigma,labels):

  batch,seq_len,mudim = mu.shape
  batch,seq_len,sigmadim = sigma.shape

  assert mudim==sigmadim

  mu = mu.view(batch*seq_len,mudim)
  sigma = sigma.view(batch*seq_len,sigmadim)
  labels = labels.view(batch*seq_len,1)

  gauss_prices = torch.distributions.Normal(mu,sigma)
  total_likelihood = -1 * torch.sum(gauss_prices.log_prob(labels))/(batch*seq_len)

  return total_likelihood

In [25]:
def accuracy_RMSE_train(mu: torch.Tensor, labels: torch.Tensor):
  predictions = mu.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [26]:
def accuracy_RMSE_test(predictions: torch.Tensor, labels: torch.Tensor):
  predictions = predictions.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [27]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sqrt((1/(N)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result


In [28]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [29]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        
        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
        
  rmse_val = rmse(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])
  nd_val = ND(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])

  # print("overall mu", overall_mu[:, params['conditioning_period']:].shape)
  # print("overall label", overall_label.shape)

  return rmse_val, nd_val







In [35]:
# rmse = accuracy_RMSE_train(overall_mu, overall_label)
# print("rmse ", rmse)

In [30]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])


In [ ]:
validate(model)

(0.2377735869947597, 0.9873128)

# TRAIN

In [31]:
model_version='PLOT_DEBUG.pt'
epochs = 1

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)

print(model)

for epoch in range(0, epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0

    if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
        # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

        checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch'] + 1

    for i, (ip_covariate, op_label_list) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        op_label_list     = [op_label.cuda() for op_label in op_label_list]

        mu, sigma, ht_ct = model(x=ip_covariate, h0_c0=None)

        # print('mu shape', mu.shape, 'sigma shape', sigma.shape, 'op label', op_label.shape)
        # loss = [loss_fn(mu, sigma, op_label) * params['horizon_weights'] for op_label in op_label_list]

        loss = 0
        for i, op_label in enumerate(op_label_list):
          loss +=  loss_fn(mu, sigma, op_label) * params['horizon_weights'][i]

        total_loss += float(loss)
        loss.backward()
        optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    val_rmse, val_nd= validate(model=model)
    scheduler.step(val_rmse)

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss/len(train_loader),
            },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
  

    if val_rmse < best_dev_rmse:
      best_dev_rmse = val_rmse
      torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'val_rmse': val_rmse,
              'loss': total_loss/len(train_loader),
              },  f'/content/drive/MyDrive/DeepARExperiments/best_dev_acc_{best_dev_rmse}_{model_version}')
    
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse, "nd is", val_nd)


Network(
  (embedding): Embedding(5, 64)
  (lstm): LSTM(67, 128, num_layers=3, batch_first=True)
  (distribution_premu): Linear(in_features=128, out_features=1, bias=True)
  (distribution_mu): GELU()
  (distribution_presigma): Linear(in_features=128, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)


Epoch 26/1: Train Loss -0.2201, Learning Rate 0.0003
rmse is  0.21890213584245266 nd is 0.8745263


In [ ]:
# checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/best_dev_acc_0.030142039248817844_deepar_model_no_volume_v1.pt')
# model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [32]:

def get_pred_sigma_mu_labels(model):

  model.eval()
  model.cuda()


  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        
        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)

  return overall_mu, overall_sigma, overall_label

In [40]:
# import pandas as pd
# import plotly.graph_objects as go
# def plot_test_output_data(actual,predicted, sigma, window_id):
#   x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
#   df = pd.DataFrame()
#   df['actual'] = actual[window_id]
#   df['predicted'] = predicted[window_id]
#   df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
#   df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

#   fig = go.Figure()
#   fig.add_vline(x=168)
#   fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
#   fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

#   fig.add_traces(go.Scatter(
#         name='Upper Bound',
#         x=x.tolist(),
#         y=df['predicted_upper'],
#         mode='lines',
#         marker=dict(color="#444"),
#         line=dict(width=0),
#         showlegend=False
#     ))
#   fig.add_traces(
#     go.Scatter(
#         name='Lower Bound',
#         x=x.tolist(),
#         y= df['predcited_lower'],
#         marker=dict(color="#444"),
#         line=dict(width=0),
#         mode='lines',
#         fillcolor='rgba(68, 68, 68, 0.3)',
#         fill='tonexty',
#         showlegend=False
#     ))
  

#   fig.update_xaxes(title_text='Time Samples')
#   fig.update_yaxes(title_text='Normalized Stock Opening Price')
#   fig.show()

In [33]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted, sigma, window_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
  df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [41]:
# #s for the paper 3500 420
# model_no_volume = Network(params) 
# checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/deepar_model_no_volume_v1.pt')
# model_no_volume.load_state_dict(checkpoint['model_state_dict'])

model_no_volume = model

In [36]:
mu_no_volume, sigma_no_volume, label_no_volume =  get_pred_sigma_mu_labels(model)

# 420
# 5000
# 7000
# 11000
# 9900
# 9613
# 7105
# 2541
# 9632
# 63421
# 52132
# 57031
# 58031
# 67031
# 7031
# 9921
# 69000

# plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 1202)

In [35]:
# actual,predicted, sigma, window_id = label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 1202

# x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
# df = pd.DataFrame()
# df['actual'] = actual[window_id]
# df['predicted'] = predicted[window_id]
# df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
# df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

# fig = go.Figure()
# fig.add_vline(x=168)
# fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
# fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

# fig.add_traces(go.Scatter(
#       name='Upper Bound',
#       x=x.tolist(),
#       y=df['predicted_upper'],
#       mode='lines',
#       marker=dict(color="#444"),
#       line=dict(width=0),
#       showlegend=False
#   ))
# fig.add_traces(
#   go.Scatter(
#       name='Lower Bound',
#       x=x.tolist(),
#       y= df['predcited_lower'],
#       marker=dict(color="#444"),
#       line=dict(width=0),
#       mode='lines',
#       fillcolor='rgba(68, 68, 68, 0.3)',
#       fill='tonexty',
#       showlegend=False
#   ))


# fig.update_xaxes(title_text='Time Samples')
# fig.update_yaxes(title_text='Normalized Stock Opening Price')
# fig.show()

NameError: ignored

In [37]:
actual,predicted, sigma, window_id = label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 1202

x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
df = pd.DataFrame()
df['actual'] = actual[window_id]
df['predicted'] = predicted[window_id]
df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

# frames_animated = [go.Frame(data=[go.Scatter(x=[i for i in range(x_max)], y=[df['predicted'][i] for i in range(x_max)]),
#                                   go.Scatter(x=[i for i in range(x_max)], y=[df['actual'][i] for i in range(x_max)])]) for x_max in range(params['conditioning_period']+1, 193)]

frames_animated = [go.Frame(data=[go.Scatter(x=[i for i in range(x_max)], y=[df['predicted'][i] for i in range(x_max)], name='Predicted', showlegend=True),
                                  go.Scatter(x=[i for i in range(x_max)], y=[df['actual'][i] for i in range(x_max)], name='Actual', showlegend=True),
                                  go.Scatter(x=[i for i in range(x_max)], y=[df['predicted_upper'][i]  for i in range(x_max)], mode='lines', name='Upper Bound', marker=dict(color="#444"), line=dict(width=0), showlegend=False),
                                  go.Scatter(x=[i for i in range(x_max)], y=[df['predcited_lower'][i]  for i in range(x_max)], mode='lines', name='Lower Bound', marker=dict(color="#444"), line=dict(width=0), showlegend=False, fillcolor='rgba(68, 68, 68, 0.3)', fill='tonexty')]) for x_max in range(params['conditioning_period']+1, params['conditioning_period'] + params['prediction_period'] + 1)]


fig = go.Figure(
    data=[go.Scatter(x=[i for i in range(0, params['conditioning_period'])], y=df['predicted'][0:params['conditioning_period']], name='Predicted', showlegend=True),
          go.Scatter(x=[i for i in range(0, params['conditioning_period'])], y=df['actual'][0:params['conditioning_period']], name='Actual', showlegend=True),
          go.Scatter(x=[i for i in range(0, params['conditioning_period'])], y=df['predicted_upper'][0:params['conditioning_period']], mode='lines', name='Upper Bound', marker=dict(color="#444"), line=dict(width=0), showlegend=False),
          go.Scatter(x=[i for i in range(0, params['conditioning_period'])], y=df['predcited_lower'][0:params['conditioning_period']], mode='lines', name='Lower Bound', marker=dict(color="#444"), line=dict(width=0), showlegend=False, fillcolor='rgba(68, 68, 68, 0.3)', fill='tonexty')],
    layout=go.Layout(
        xaxis=dict(range=[0, 191], autorange=False),
        yaxis=dict(range=[-1, 1], autorange=False),
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=frames_animated
)

fig.add_vline(x=167)

# fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 5
fig.update_xaxes(title_text='Time Samples')
fig.update_yaxes(title_text='Normalized Stock Opening Price')
fig.show()

fig.write_html("plot_one_saved.html")

In [1]:
! pip install gif

In [2]:
! pip install "gif[altair]"     
! pip install "gif[matplotlib]"
! pip install "gif[plotly]"

     |████████████████████████████████| 89 kB 3.0 MB/s 
     |████████████████████████████████| 844 kB 11.5 MB/s 
     |████████████████████████████████| 968 kB 4.5 MB/s 
     |████████████████████████████████| 359 kB 34.0 MB/s 
     |████████████████████████████████| 138 kB 31.6 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 35.2 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [50]:
import gif
import plotly.graph_objects as go


@gif.frame
def plot_test_output_data(actual,predicted, sigma, window_id, max_seq_len):
  x=np.linspace(1, 192, num=192)
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
  df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

  fig = go.Figure()
  fig.add_vline(x=167)
  fig.add_vline(x=191)
  fig.add_traces(go.Scatter(x=x, y = df['actual'][0:max_seq_len], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'][0:max_seq_len], mode = 'lines', name = 'Predicted'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'][0:max_seq_len],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'][0:max_seq_len],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  
  return fig

frames = []
EG_ID = 3200
for i in range(167, 193):
    frame = plot_test_output_data(actual,predicted, sigma, EG_ID, i)
    frames.append(frame)

gif.save(frames, 'gif_v2.gif', duration=100)



In [38]:
import random
import plotly.graph_objects as go
import pandas as pd
import gif

df = pd.DataFrame({
    't': list(range(10)) * 10,
    'x': [random.randint(0, 100) for _ in range(100)],
    'y': [random.randint(0, 100) for _ in range(100)]
})

@gif.frame
def plot(i):
    d = df[df['t'] == i]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=d["x"],
        y=d["y"],
        mode="markers"
    ))
    fig.update_layout(width=500, height=300)
    return fig

frames = []
for i in range(10):
    frame = plot(i)
    frames.append(frame)

gif.save(frames, 'example.gif', duration=100)


In [44]:
frames = []
for i in range(167, 193):
    frame = plot_test_output_data(actual,predicted, sigma, window_id, i)
    frames.append(frame)

gif.save(frames, 'example.gif', duration=20)


In [132]:
plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 1202, 120)

In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result

print("rmse ", rmse(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

print("nd ", ND(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

rmse  0.06767744374120029
nd  0.057781804


In [ ]:
from google.colab import drive
drive.mount('/content/drive')